In [ ]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

In [ ]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
# url = "https://ethresear.ch/t/ahead-of-time-block-auctions-to-enable-execution-preconfirmations/21345"
url = "https://ethresear.ch/t/based-rollups-superpowers-from-l1-sequencing/15016"

response = requests.get(url)

soup = BeautifulSoup(response.content, "html.parser")

article = soup.find("div", {"class": "post"}).get_text()

In [ ]:
article

'\nTLDR: We highlight a special subset of rollups we call “based” or “L1-sequenced”. The sequencing of such rollups—based sequencing—is maximally simple and inherits L1 liveness and decentralisation. Moreover, based rollups are particularly economically aligned with their base L1.\nDefinition\nA rollup is said to be based, or L1-sequenced, when its sequencing is driven by the base L1. More concretely, a based rollup is one where the next L1 proposer may, in collaboration with L1 searchers and builders, permissionlessly include the next rollup block as part of the next L1 block.\nAdvantages\n\n\nliveness: Based sequencing enjoys the same liveness guarantees as the L1. Notice that non-based rollups with escape hatches suffer degraded liveness:\n\n\nweaker settlement guarantees: Transactions in the escape hatch have to wait a timeout period before guaranteed settlement.\n\ncensorship-based MEV: Rollups with escape hatches are liable to toxic MEV from short-term sequencer censorship during

In [ ]:
# article = article.replace('.', '.<eos>')
article = article.replace('.\n', '.<eos>')
article = article.replace('?', '?<eos>')
article = article.replace('!', '!<eos>')

In [ ]:
article

'\nTLDR: We highlight a special subset of rollups we call “based” or “L1-sequenced”. The sequencing of such rollups—based sequencing—is maximally simple and inherits L1 liveness and decentralisation. Moreover, based rollups are particularly economically aligned with their base L1.<eos>Definition\nA rollup is said to be based, or L1-sequenced, when its sequencing is driven by the base L1. More concretely, a based rollup is one where the next L1 proposer may, in collaboration with L1 searchers and builders, permissionlessly include the next rollup block as part of the next L1 block.<eos>Advantages\n\n\nliveness: Based sequencing enjoys the same liveness guarantees as the L1. Notice that non-based rollups with escape hatches suffer degraded liveness:\n\n\nweaker settlement guarantees: Transactions in the escape hatch have to wait a timeout period before guaranteed settlement.<eos>\ncensorship-based MEV: Rollups with escape hatches are liable to toxic MEV from short-term sequencer censorsh

In [ ]:
max_chunk = 500

In [ ]:
sentences = article.split('<eos>')
current_chunk = 0
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1:
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

0


In [ ]:
len(chunks)

2

In [ ]:
res = summarizer(chunks, max_length=120, min_length=30, do_sample=False)

In [ ]:
res[0]

{'summary_text': ' We highlight a special subset of rollups we call “based” or “L1-sequenced” The sequencing of such rollups is maximally simple and inherits L1 liveness and decentralisation . Based rollups are particularly economically aligned with their base L1 .'}

In [ ]:
text = ' '.join([summ['summary_text'] for summ in res])

In [ ]:
text

' We highlight a special subset of rollups we call “based” or “L1-sequenced” The sequencing of such rollups is maximally simple and inherits L1 liveness and decentralisation . Based rollups are particularly economically aligned with their base L1 .  The name “based rollup” derives from the close proximity with the base L1 . The winning rollup may leverage the improved security, decentralisation, simplicity, and alignment of based rollups to achieve dominance and ultimately maximise revenue .'

In [ ]:
with open('blogsummary.txt', 'w') as f:
    f.write(text)

# get new urls

In [ ]:
import requests
import json

# fucntion for getting the urls from the json file
def urls():
    try:
        with open('latest.json', 'r') as f:
            data = json.load(f)
    except FileNotFoundError:
        return "No data found."

    urls = []
    for topic in data["topic_list"]["topics"]:
        custom_url = f"https://ethresear.ch/t/{topic['slug']}/{topic['id']}"
        urls.append(custom_url)

    return urls

# Function for getting the new blog URLs
def get_urls(new_posts=False, update_json=False):
    if not new_posts:
        return urls()

    url = "https://ethresear.ch/latest.json"
    response = requests.get(url)
    latest_data = response.json()

    try:
        with open('latest.json', 'r') as f:
            previous_data = json.load(f)
    except FileNotFoundError:
        previous_data = {"topic_list": {"topics": []}}

    latest_topics = {topic["id"]: topic for topic in latest_data["topic_list"]["topics"]}
    previous_topics = {topic["id"]: topic for topic in previous_data["topic_list"]["topics"]}
    new_blogs = [topic for topic_id, topic in latest_topics.items() if topic_id not in previous_topics]

    new_urls = []
    if new_blogs:
        for blog in new_blogs:
            custom_url = f"https://ethresear.ch/t/{blog['slug']}/{blog['id']}"
            new_urls.append(custom_url)

    if update_json:
        with open('latest.json', 'w') as f:
            json.dump(latest_data, f)

    return new_urls

for urls in get_urls():
    print(urls)